**ÉTAPE 0** : préparation des données

In [3]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True) ## ces deux lignes sont fondamentales, ne pas les retirer !!
df.set_index('ticker', inplace=True)


There are no NaN values in the dataframe


In [4]:
import process 
lookback_window = [5000, 5250]
process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)

ticker,AA,ABM,ABT,ADI,ADM,ADX,AEE,AEG,AEM,AEP,...,XLI,XLK,XLP,XLU,XLV,XLY,XOM,XRX,YUM,ZTR
ticker,,,,,,,,,,,,,,,,,,,,,
AA,1.000000,0.210713,0.066335,0.304119,0.386047,0.238990,-0.002054,0.372975,0.081120,-0.016727,...,0.452733,0.217769,0.135824,0.120502,0.091232,0.331624,0.497741,0.374066,0.260370,0.155580
ABM,0.210713,1.000000,0.266627,0.298192,0.505141,0.514753,0.378482,0.485270,0.163598,0.417213,...,0.502062,0.413210,0.541022,0.457376,0.445494,0.430725,0.433626,0.473183,0.266579,0.404355
ABT,0.066335,0.266627,1.000000,0.292098,0.404411,0.496111,0.493633,0.265322,0.100852,0.513681,...,0.496866,0.581703,0.605572,0.535450,0.740983,0.414775,0.191937,0.105420,0.278291,-0.005929
ADI,0.304119,0.298192,0.292098,1.000000,0.438682,0.609714,0.363606,0.439790,0.144624,0.205188,...,0.555041,0.611330,0.433946,0.392778,0.499632,0.625127,0.375675,0.218490,0.299477,0.338722
ADM,0.386047,0.505141,0.404411,0.438682,1.000000,0.594137,0.440526,0.570632,0.182628,0.438031,...,0.719256,0.537401,0.688310,0.557074,0.547545,0.521051,0.546795,0.419870,0.330139,0.253729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,0.331624,0.430725,0.414775,0.625127,0.521051,0.758335,0.439081,0.488496,0.179463,0.376045,...,0.750748,0.816325,0.594544,0.531203,0.664660,1.000000,0.454554,0.364647,0.583761,0.300539
XOM,0.497741,0.433626,0.191937,0.375675,0.546795,0.497171,0.344984,0.538250,0.154030,0.335934,...,0.613797,0.377444,0.417391,0.460625,0.403054,0.454554,1.000000,0.512675,0.296081,0.312790
XRX,0.374066,0.473183,0.105420,0.218490,0.419870,0.452733,0.072198,0.462261,0.286879,0.171540,...,0.504920,0.274119,0.254480,0.211230,0.213860,0.364647,0.512675,1.000000,0.346935,0.454572
